# Unit-testing and testing a Lattice-Boltzman simulation

## Introduction


The Lattice-Boltzmann method solves the Navier-Stokes equations - i.e. fluid
dynamics. This method maps the continuous fluid dynamics problem onto a discrete cartesian grid where populations of fictives populations stream (move from point to point) and collide (intereact and exchange particles). Each population of particles moves at a distinct speed: for instance, there is one population of particles moving one grid point East each turn, and another moving one grid point North-West.

In this chapter, we will describe the test driven implementation of Lattice-Boltzmann code simulation in [Julia](http://julialang.org/). We will put special emphasis on two approaches for designing tests. In practice, both of these approaches have a common goal; it is to simplify how we think about a piece of code.

- *Mechanical* tests that simply check a function does what it does, whithout any regards or care for actual physics. The code to test is transparent box with it inside mechanics visible. However, its input and output need not be physically meaningful (during testing), as long as the internal mechanics get exercised the same way. This may mean that we will substitute real numbers with dummy integers in order to avoid pesky floating point comparisons. Or we may substitute pieces of the internals with dummy agents: we care that internal function X gets called twice and that its arguments are such and such according to the flow of data within a piece of code, but we often may not care whether function X does anything! (why? most likely because the real function X is tested elsewhere).
- *Domain-area* tests that try and implement specific situations with known outcomes from the domain-area of the code, namely physics here. The code to test is a black box that is fed physical inputs, and thanks to whichever wizard with whatever magic, outputs more physics. In that case, we will often try and reproduces expected behaviours rather than reproduce absolute numbers; e.g . what happens to the velocity when the pressure is doubled, or the size of the pipe halved. Going after specific behaviors is often conceptually simpler that attempting to check that every aspect of the state of a simulation is correct.

The code used throughout the chapter is based on
[LatBo.jl](https://github.com/UCL/LatBo.jl). It was developped during the
course of a Hackathon sponsored by the [Software Sustainability
Institute](http://www.software.ac.uk/).

[LatBo.jl](https://github.com/UCL/LatBo.jl) exists as a Julia package. It can
be installed from the Julia prompt with:

In [1]:
if !haskey(Pkg.installed(), "LatBo")
    Pkg.clone("git@github.com:UCL/LatBo.jl.git", "close_enough")
end

And since this about testing, we should certainly check that all tests pass.  First, we will add  [FactCheck](https://github.com/JuliaLang/FactCheck.jl),  a  package that facilitates writing tests.

In [2]:
Pkg.add("FactCheck")

INFO: Nothing to be done
INFO: METADATA is out-of-date — you may not have the latest version of FactCheck
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [3]:
Pkg.test("LatBo")

INFO: Computing test dependencies for LatBo...
INFO: No packages to install, update or remove
INFO: Testing LatBo


Physical to and from LB units using SIUnits
     - P to LB
     - LB to P
     - P to LB to P
19 facts verified.
Physical to and from LB units
29 facts verified.
Constructing playground with a single pipe
     - 2d
     - 3d
311 facts verified.
Check pipe geometries
     - 2d
     - 3d
160 facts verified.
Check half-plane geometries
     - 3d
128 facts verified.
Check sphere geometries
     - 3d
512 facts verified.
Grid coordinates to array index
     - Cartesian
     - Periodic
9 facts verified.
Array index to grid coordinates
3 facts verified.
Compute neighbor index
10 facts verified.
Lattice direction inversion
     - for D2Q9
     - for D3Q19
8 facts verified.
Compute neighbor index from sim/lattice object
20 facts verified.
Thermodynamic quantities and functions
     - for D2Q9
       - rho is a geometric series
       - homogeneous populations sum to zero momentum
       - velocity from momentum and density 
       - equilibrium function
       - local quantities aggregator
     

INFO: LatBo tests passed
INFO: No packages to install, update or remove


## Description of the Lattice-Boltzmann model

As mentioned previously, Lattice-Boltzmann works by modeling the interactions of distinct fictitious populations of particles on a square grid. At time $t$, on each site $\vec{r}$ of the grid, there $f_i(t, \vec{r})$ particles of type $i$. Each type of particle moves at a specific speed in a specific direction (velocity $\vec{c}_i$), never wavering.

There are three steps to the algorithm:

### Initialization

The $f_i(t, \vec{r})$ are set such in some manner that ensure convergence. We will not examine this step further.

### Collision

In this step, all the particles at each site $\vec{r}$ collide together. Some particles of type $1$ will become particles of type $2$, and vice-versa. In practice, this means the collision change the speed and direction of the particles. In one approximation, used in this code, the new numbers of particle $f^{*}_i(t, \vec{r})$ is obtained as:

$f^{*}_i(t, \vec{r}) = f_i(t, \vec{r}) + \frac{1}{\tau}\left[f_i(t, \vec{r}) - f^{eq}_i(...)\right]$

$f^{eq}_i(...)$ is the equilibrium number of particles of type $i$. It is obtained from the fluid velocity and density at site $\vec{r}$ in ways we shall not fathom here.

### Streaming

Streaming means the particles move to the next site, depending upon the speed of each particle. *After collision*, the particles of type $i$ on site $\vec{r}$ move in concert at the speed $c_i$ characteristic of their type to the neighbouring site $\vec{r}'=\vec{r} + \delta\,t\vec{c}_i$, where $\delta\,t$ is the timestep. And so:

$f_i(t+1, \vec{r}) = f_i^{*}(t, \vec{r} - \delta\,t\vec{c}_i)$

There is one caveat however. What happens at the edge of the simulation? This is determined by the boundary conditions. In some cases, the particles will hit a wall and bounce back. In others, the particles will reach an inlet or an outlet, and something happens that will ensure (for instance) the fluid streams into the simulation with a given velocity, and streams out at a given pressure. The populations are then adjusted explicitly to constrain to the boundary condition. 

#### Content:
- [Mechanical Tests](Mechanical Tests.ipynb)
- [Domain-area Tests](Physical Tests.ipynb)